In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from google.colab import drive
from numpy.random import seed

In [ ]:
drive.mount('/gdrive')
%cd /gdrive/MyDrive/F21DL_CW2/Data/CSV

In [ ]:
X_train = pd.read_csv('x_train_gr_smpl.csv')
X_test = pd.read_csv('x_test_gr_smpl.csv')

X = pd.concat([X_train, X_test]).reset_index().drop(columns = ['index'])

y_train = pd.read_csv('y_train_smpl.csv')
y_test = pd.read_csv('y_test_smpl.csv')

y = pd.concat([y_train, y_test]).reset_index().drop(columns = ['index'])

X_train_4000, X_test_4000, y_train_4000, y_test_4000 = train_test_split(X, y, test_size=0.55, random_state=42, stratify = y)
X_train_9000, X_test_9000, y_train_9000, y_test_9000 = train_test_split(X, y, test_size=0.95, random_state=42, stratify = y)

X_train_4000 = X_train_4000.reset_index().drop(columns = ['index'])
X_test_4000 = X_test_4000.reset_index().drop(columns = ['index'])
y_train_4000 = y_train_4000.reset_index().drop(columns = ['index'])
y_test_4000 = y_test_4000.reset_index().drop(columns = ['index'])

X_train_9000 = X_train_9000.reset_index().drop(columns = ['index'])
X_test_9000 = X_test_9000.reset_index().drop(columns = ['index'])
y_train_9000 = y_train_9000.reset_index().drop(columns = ['index'])
y_test_9000 = y_test_9000.reset_index().drop(columns = ['index'])

X_train_tensor = tf.reshape(np.array(X_train), shape = (9690, 48, 48))
X_test_tensor = tf.reshape(np.array(X_test), shape = (3090, 48, 48))
X_train_4000_tensor = tf.reshape(np.array(X_train_4000), shape = (5750, 48, 48))
X_test_4000_tensor = tf.reshape(np.array(X_test_4000), shape = (7030, 48, 48))
X_train_9000_tensor = tf.reshape(np.array(X_train_9000), shape = (639, 48, 48))
X_test_9000_tensor = tf.reshape(np.array(X_test_9000), shape = (12141, 48, 48))

y_train_0 = pd.read_csv('y_train_smpl_0.csv')
y_test_0 = pd.read_csv('y_test_smpl_0.csv')

y_train_1 = pd.read_csv('y_train_smpl_1.csv')
y_test_1 = pd.read_csv('y_test_smpl_1.csv')

y_train_2 = pd.read_csv('y_train_smpl_2.csv')
y_test_2 = pd.read_csv('y_test_smpl_2.csv')

y_train_3 = pd.read_csv('y_train_smpl_3.csv')
y_test_3 = pd.read_csv('y_test_smpl_3.csv')

y_train_4 = pd.read_csv('y_train_smpl_4.csv')
y_test_4 = pd.read_csv('y_test_smpl_4.csv')

y_train_5 = pd.read_csv('y_train_smpl_5.csv')
y_test_5 = pd.read_csv('y_test_smpl_5.csv')

y_train_6 = pd.read_csv('y_train_smpl_6.csv')
y_test_6 = pd.read_csv('y_test_smpl_6.csv')

y_train_7 = pd.read_csv('y_train_smpl_7.csv')
y_test_7 = pd.read_csv('y_test_smpl_7.csv')

y_train_8 = pd.read_csv('y_train_smpl_8.csv')
y_test_8 = pd.read_csv('y_test_smpl_8.csv')

y_train_9 = pd.read_csv('y_train_smpl_9.csv')
y_test_9 = pd.read_csv('y_test_smpl_9.csv')

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
seed(42)

In [ ]:
for train, test in kfold.split(X_train, y_train):
  X_train_tensor[train]

TypeError: ignored

In [ ]:
fold_no = 1
acc_per_fold_cnn = list()
class_report_per_fold_cnn = list()
auc_per_fold_cnn = list()
conf_per_fold_cnn = list()

for train, test in kfold.split(X_train, y_train):

    # Define the model architecture
    model_cnn_basic = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(128, (4,4), activation='relu', input_shape=(48, 48, 1)),
      tf.keras.layers.MaxPooling2D(3, 3),
      tf.keras.layers.Conv2D(128, (4,4), activation='relu'),
      tf.keras.layers.MaxPooling2D(3,3),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    # Compile the model
    model_cnn_basic.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Generate a print
    # print('------------------------------------------------------------------------')
    # print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model_cnn_basic.fit(X_train_tensor[train], y_train.iloc[train, :],epochs=300)

    # Generate generalization metrics
    scores = model_cnn_basic.evaluate(X_train_tensor[test], y_train.iloc[test, :], verbose=0)
    
    acc_per_fold_cnn.append(scores[1] * 100)
    y_pred = model_cnn_basic.predict(X_train_tensor[test], batch_size=64, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    class_report_per_fold_cnn.append(pd.DataFrame(classification_report(y_train.iloc[test, :], y_pred_bool, output_dict=True)).T)
    auc_p_c = list()
    for i in range(10):
      fpr, tpr, _ = roc_curve(np.array(y_train.iloc[test, :]), y_pred_bool, pos_label=i)
      auc_p_c.append(auc(fpr, tpr))
    auc_per_fold_cnn.append(auc_p_c)
    conf_per_fold_cnn.append(tf.math.confusion_matrix(y_train.iloc[test, :], y_pred_bool))

    # Increase fold number
    fold_no = fold_no + 1